## Data Wrangling: Clean, Transform, Merge, Reshape 
##### - Much of the programming work in data analysis and modeling is spent in data preparation.
##### - That is, data loading, cleaning, transforming, and rearranging.
##### - This is discussed and demonstrated below.

In [4]:
# Importing the necessary libraries 

import pandas as pd 
import numpy as np 